In [10]:
import spacy
from sklearn.svm import LinearSVC

parser = spacy.load('en_core_web_md')

In [11]:
import pandas as pd
import numpy as np

In [12]:
TRAIN = pd.read_csv('../datasets/travel train.csv')
TEST = pd.read_csv('../datasets/travel test.csv')

In [13]:
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

In [14]:
def tokenizeText(sample):
    tokens = parser(sample)
    return tokens.vector
    
#     tokens = parser(sample)
#     lemmas = []
#     for tok in tokens:
#         lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
#     tokens = lemmas
#     #tokens = [word.lemma_ for word in tokens if word.lemma_ != "-PRON-"]
#     #tokens = [tok for tok in tokens if tok not in set(SYMBOLS)|STOP_WORDS]
    
#     while "" in tokens:
#         tokens.remove("")
#     while " " in tokens:
#         tokens.remove(" ")
#     while "\n" in tokens:
#         tokens.remove("\n")
#     while "\n\n" in tokens:
#         tokens.remove("\n\n")

#     return tokens

In [15]:
def append_vectors(row):
    sample = row[0]
    vector = tokenizeText(sample)
    size = len(vector)
    row['vector'] = vector
    #row['size'] = size
    return row


In [16]:
def featurize(dataset):
    dataset = dataset.apply(append_vectors, axis = 1)
    return dataset


In [17]:
training_data = featurize(TRAIN)
training_data.head()

,text,intent,vector
0,how do I get to jordan,gotoPlace,"[-0.128587, 0.242902, -0.291153, -0.118759, 0...."
1,flight Mumbai,gotoPlace,"[-0.28538, -0.04083, 0.40544, -0.02119, 0.6039..."
2,i am in india. I want to go to jordan.,gotoPlace,"[-0.0341223, 0.290647, -0.114755, -0.143871, 0..."
3,book tickets from bangalore to jordan for two ...,gotoPlace,"[-0.180983, 0.138287, 0.00821977, -0.0843011, ..."
4,Jordan flight tickets,gotoPlace,"[-0.10317, 0.0151367, 0.35412, -0.130847, 0.51..."


In [18]:
X_train = training_data['vector']
y_train = training_data['intent']

In [19]:
clf = LinearSVC()
X_train = np.array(list(X_train), dtype=np.float)


In [20]:
clf.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [21]:
def append_svc_response(row):
    utterance = row['vector']
    ip = utterance.reshape(1,-1)
    #ip = np.array(list(utterance),dtype = np.float)
    #ip = pd.Series(utterance)
    
    response = clf.predict(ip)[0]
    
    try:
        row['predicted'] = response
        """
        if response[2]:
            row['predicted_score'] = response[2]
        else:
            row['predicted_score'] = 0
        """
        
        
        #print("q = {}, ans = {}" .format(utterance,response))
            
    except Exception as e:
        row['predicted'] = None
        #row['predicted_score'] = 0
        print("q = {}, ans = {}" .format(utterance,None))
    
    return row

In [22]:
def label_correct_responses(row):
    expected = row['expected intent']
    predicted = row['predicted']
    
    if expected == predicted:
        row['Correct'] = 1
    else:
        row['Correct'] = 0
    
    return row

In [23]:
def svc_test(test, write = False):
    test = test.apply(append_svc_response, axis = 1)
    test = test.apply(label_correct_responses, axis = 1)
    print(test['Correct'].mean())
    
    if write:
        test.to_csv('Word2vec with SVC.csv')
    
    return test


In [24]:
test_featurized = featurize(TEST)
#test_featurized

test_output = svc_test(test_featurized, False)
test_output

0.844827586207


,utterance,expected intent,"difficulty (1 = easy, 3 = hard)",vector,predicted,Correct
0,go to singapore,gotoPlace,1,"[0.241557, 0.145168, -0.11223, -0.0375067, 0.4...",gotoPlace,1
1,I want to travel to budapest,gotoPlace,1,"[0.209776, 0.160227, -0.123439, -0.0127925, 0....",gotoPlace,1
2,book me two flight tickets to JFK,gotoPlace,2,"[0.121406, -0.0428671, -0.00398028, 0.00568943...",gotoPlace,1
3,cheap flights to South Korea,gotoPlace,1,"[0.106686, -0.101297, 0.141946, -0.040256, 0.7...",gotoPlace,1
4,Book me a first class ticket from Mumbai to Du...,gotoPlace,1,"[0.0100925, 0.186582, -0.0167053, 0.0310381, 0...",gotoPlace,1
5,how do i reach Vietnam?,gotoPlace,3,"[-0.0460825, 0.250176, -0.240383, -0.238976, 0...",placeExplore,0
6,cheapest flight from Bangalore to Kolkata,gotoPlace,1,"[-0.260177, 0.0339608, 0.157403, 0.00194834, 0...",gotoPlace,1
7,is there a ticket for Hyderabad on Monday?,gotoPlace,1,"[-0.114622, 0.251836, 0.0409536, -0.0574417, 0...",placeLogistics,0
8,good hotels in mumbai?,placeAccommodation,1,"[-0.179469, 0.236357, 0.20577, -0.423971, 0.50...",placeAccommodation,1
9,hotel bookings for Mumbai next month,placeAccommodation,2,"[-0.117477, 0.275067, 0.162492, -0.296618, 0.4...",placeAccommodation,1


In [25]:
def predict(utterance):
    row = pd.DataFrame({'utterance':[utterance]})
    row = featurize(row)
    test_input = np.array(list(row['vector']), dtype=np.float)
    response = clf.predict(test_input)[0]
    return response

# r = pd.DataFrame({'utterance':[utterance]})
# r = featurize(r)
# #ip = pd.Series(r['vector'])
# ip = np.array(list(r['vector']), dtype=np.float)

# clf.predict(ip)
# #response = clf.predict(ip)[0]
# #response

In [26]:
utterance = "places to eat"
predict(utterance)

'placeExplore'

In [27]:
test_output.to_csv('Word2vec with SVC.csv')